This sheet is playing around with tensorflow, trying to figure out how to manage experimentation from an engineering perspective.  I want to be able to do a lot of runs, tweaking all kinds of things, without having to context-switch to write down the results.  I want the results to be written down for me.

This doesn't get all the way there, but it starts to point the way.

In [1]:
import tensorflow as tf
import datetime
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# A local variable for keeping track of various runs
# TODO: Figure out how to pickle this.
runs = []

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [40]:
def run(hyperparams, model_params, model, output, train_data, validation_data, test_data):
    """Run the model and store some data about it.
    """
    # Parse input.
    step_size_map = hyperparams['step_size_map']
    num_batches = hyperparams['num_batches']
    batch_size = hyperparams['batch_size']
    num_outputs = hyperparams['num_outputs']
    
    x = model_params['x']
    y_ = model_params['y_']
    W = model_params['W']
    b = model_params['b']
    y = model_params['y']
    
    loss = model['loss']
    train_step = model['train_step']
    
    accuracy = output['accuracy']
    
    # Train the model
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for _ in range(num_batches):
            # Adjust the step size
            if _ in step_size_map:
                step_size = step_size_map[_]
                train = train_step(step_size)
                print(f'Step size: {step_size}')
            
            # Load a batch, and train one step it.
            xs, ys = train_data.next_batch(batch_size)
            sess.run(train, feed_dict={x: xs, y_: ys})
            
            # Periodic output
            if (_ + 1) % (num_batches // num_outputs) == 0:
                xs, ys = validation_data.images, validation_data.labels
                vld_accuracy = sess.run(accuracy, feed_dict={x: xs, y_: ys})
                print(f'Step {_ + 1} accuracy: {vld_accuracy:.3f}')
                vld_loss = sess.run(loss, feed_dict={x:xs, y_: ys})
                print(f'Step {_ + 1} loss: {vld_loss:.3f}')
    
        test_accuracy = sess.run(accuracy, feed_dict={x: test_data.images, y_: test_data.labels})
        print(f'Test accuracy: {test_accuracy:.3f}')
        test_loss = sess.run(loss, feed_dict={x: test_data.images, y_: test_data.labels})
        print(f'Test loss: {test_loss:.3f}')
        
    runs.append({
        'time': datetime.datetime.now(),
        'hyperparams': hyperparams,
        'model_params': model_params,
        'model': model,
        'output': output,
        'accuracy': test_accuracy,
    })

In [41]:
xdim = mnist.train.images[0].size
xtype = mnist.train.images[0].dtype
ydim = mnist.train.labels[0].size

x = tf.placeholder(xtype, [None, xdim])
y_ = tf.placeholder(xtype, [None, ydim]) 
W = tf.Variable(tf.zeros([xdim, ydim]))
b = tf.Variable(tf.zeros([ydim]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=tf.matmul(x, W) + b))
#loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = lambda _: tf.train.GradientDescentOptimizer(_).minimize(loss)

model_params = {
    'x': x,
    'y_': y_,
    'W': W,
    'b': b,
    'y': y,
}

model = {
    'loss': loss, 
    'train_step': train_step,
}

output = {
    'accuracy': tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)), tf.float32)),
}

In [42]:
hyperparams = {
    'step_size_map': {0: 3, 2500: 0.3},
    'num_batches': 6000,
    'batch_size': 100,
    'num_outputs': 10,
}

In [43]:
run(hyperparams, model_params, model, output, mnist.train, mnist.validation, mnist.test)

Step size: 3
Step 600 accuracy: 0.919
Step 600 loss: 1.556
Step 1200 accuracy: 0.921
Step 1200 loss: 1.547
Step 1800 accuracy: 0.926
Step 1800 loss: 1.543
Step 2400 accuracy: 0.931
Step 2400 loss: 1.540
Step size: 0.3
Step 3000 accuracy: 0.930
Step 3000 loss: 1.539
Step 3600 accuracy: 0.930
Step 3600 loss: 1.539
Step 4200 accuracy: 0.931
Step 4200 loss: 1.538
Step 4800 accuracy: 0.931
Step 4800 loss: 1.539
Step 5400 accuracy: 0.930
Step 5400 loss: 1.538
Step 6000 accuracy: 0.930
Step 6000 loss: 1.538
Test accuracy: 0.927
Test loss: 1.541


Note: the inner loop in `run()` is:
```
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(num_batches):
        # Adjust the step size
        if _ in step_size_map:
            step_size = step_size_map[_]
            train = train_step(step_size)
            print('Step size: {}'.format(step_size))

        # Load a batch, and train one step it.
        xs, ys = train_data.next_batch(batch_size)
        sess.run(train, feed_dict={x: xs, y_: ys})

        # Periodic output
        if (_ + 1) % (num_batches // num_outputs) == 0:
            vld_accuracy = sess.run(
                accuracy, 
                feed_dict={x: validation_data.images, y_: validation_data.labels})
            print('Step {} accuracy: {:.3f}'.format(_ + 1, vld_accuracy))
```

The inner-loop in the CNN MNIST [example](https://www.tensorflow.org/get_started/mnist/pros) looks like this:
```
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_batches):
        batch = mnist.train.next_batch(batch_size)
        
        # Periodic output
        if i % 100 == 0:
              train_accuracy = accuracy.eval(feed_dict={
                  x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    
        # Train the batch:
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
```
which is more-or-less identical.  This gives me hope that I can make this fairly generic.

In [57]:
def run_session(get_train_data, num_batches, train_step=None, train_step_update=None, 
                output_cnd=None, output_fn=None, final_output_fn=None):
    """
    Args:
     - get_train_data: A function which returns (features, labels) for a single batch.
     - num_batches: The number of batches to process
     - train_step: An optimizer of the form of those in `tf.train`
     - train_step_update: a function of `sess` and the current iteration number that returns an
         updated `train_step`. If this function returns `None`, `train_step` will not be updated.     
     - output_cnd: A function of the current iteration number, that returns True if we should print intermediate
         output at this iteration.
     - output_fn: A function of `sess` and the current iteration number that prints intermediate output.
     - final_output_fn: A function of `sess` that produces any final output immediately before closing the session.
    """
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for _ in range(num_batches):
            # Update training step, if necessary:
            if train_step_update is not None:
                update = train_step_update(sess, _)
                if update is not None:
                    train_step = update

            # Load and train
            xs, ys = get_train_data()
            sess.run(train_step, feed_dict={x: xs, y_: ys})
            
            # Conditional Output
            if output_cnd is not None and output_cnd(_):
                output_fn(sess, _)
                
        if final_output_fn is not None:        
            final_output_fn(sess)        

In [62]:
get_train_data = lambda: mnist.train.next_batch(hyperparams['batch_size'])
def train_step_update(sess, i): 
    return train_step(hyperparams['step_size_map'][i]) if i in hyperparams['step_size_map'] else None
def output_cnd(i):
    return (i + 1) % (hyperparams['num_batches'] // hyperparams['num_outputs']) == 0
def output_fn(sess, _):
    validation_data = mnist.validation
    xs, ys = validation_data.images, validation_data.labels
    vld_accuracy = sess.run(output['accuracy'], feed_dict={x: xs, y_: ys})
    print(f'Step {_ + 1} accuracy: {vld_accuracy:.3f}')
    vld_loss = sess.run(model['loss'], feed_dict={x:xs, y_: ys})
    print(f'Step {_ + 1} loss: {vld_loss:.3f}')
def final_output_fn(sess):
    print()
    test_data = mnist.test
    test_accuracy = sess.run(output['accuracy'], feed_dict={x: test_data.images, y_: test_data.labels})
    print(f'Test accuracy: {test_accuracy:.3f}')
    test_loss = sess.run(model['loss'], feed_dict={x: test_data.images, y_: test_data.labels})
    print(f'Test loss: {test_loss:.3f}')

In [63]:
run_session(
    get_train_data=get_train_data, 
    num_batches=hyperparams['num_batches'], 
    train_step_update=train_step_update, 
    output_cnd=output_cnd, 
    output_fn=output_fn, 
    final_output_fn=final_output_fn)

Step 600 accuracy: 0.915
Step 600 loss: 1.558
Step 1200 accuracy: 0.924
Step 1200 loss: 1.547
Step 1800 accuracy: 0.927
Step 1800 loss: 1.542
Step 2400 accuracy: 0.926
Step 2400 loss: 1.542
Step 3000 accuracy: 0.931
Step 3000 loss: 1.538
Step 3600 accuracy: 0.931
Step 3600 loss: 1.538
Step 4200 accuracy: 0.930
Step 4200 loss: 1.538
Step 4800 accuracy: 0.932
Step 4800 loss: 1.538
Step 5400 accuracy: 0.931
Step 5400 loss: 1.538
Step 6000 accuracy: 0.930
Step 6000 loss: 1.538

Test accuracy: 0.926
Test loss: 1.541


In [51]:
hyperparams['num_batches']

6000

In [64]:
tf.nn.conv2d?